<a href="https://colab.research.google.com/github/s-pike3/Projects_In_AI-ML/blob/main/Final_Project_Pet_Pawpularity_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sarah Pike

Projects in AI & ML

Final Project

This notebook contains the training process used to fine tune each of the models in my paper. It was adapted from the following notebook : https://www.kaggle.com/code/ytakayama/train-pytorch-swin-5fold-some-tips

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
#import kagglehub
#kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
#download dataset
petfinder_pawpularity_score_path = kagglehub.competition_download('petfinder-pawpularity-score')

100%|██████████| 983M/983M [00:15<00:00, 66.9MB/s]

Extracting files...


In [3]:
#import saved models from Kaggle
kozodoi_timm_pytorch_image_models_path = kagglehub.dataset_download('kozodoi/timm-pytorch-image-models')
sarahpike1_timm_fine_tuned_4_path = kagglehub.dataset_download('sarahpike1/timm-fine-tuned-4')
sarahpike1_beit_fine_tuned_4_path = kagglehub.dataset_download('sarahpike1/beit-fine-tuned-4')
sarahpike1_swin_beit_ensemble_path = kagglehub.dataset_download('sarahpike1/swin-beit-ensemble')
sarahpike1_conv_fine_tuned_4_path = kagglehub.dataset_download('sarahpike1/conv-fine-tuned-4')
sarahpike1_eva02_fine_tuned_4_path = kagglehub.dataset_download('sarahpike1/eva02-fine-tuned-4')
sarahpike1_beit_fine_tuned_4_v1_path = kagglehub.dataset_download('sarahpike1/beit-fine-tuned-4-v1')
sarahpike1_swin384_fine_tuned_path = kagglehub.dataset_download('sarahpike1/swin384-fine-tuned')

100%|██████████| 1.26M/1.26M [00:00<00:00, 106MB/s]

Extracting files...


100%|██████████| 690M/690M [00:07<00:00, 103MB/s]

Extracting files...


100%|██████████| 690M/690M [00:09<00:00, 74.1MB/s]

...

Extracting files...


100%|██████████| 304M/304M [00:03<00:00, 88.5MB/s]

Extracting files...


100%|██████████| 690M/690M [00:12<00:00, 59.9MB/s]

Extracting files...


### Setup

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore")
#general
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.preprocessing import RobustScaler
import pickle
from tqdm.auto import tqdm
from collections import defaultdict
import os
import numpy as np
import pandas as pd
import random
import gc
import cv2
gc.enable()
import glob
pd.set_option('display.max_columns', None)
from sklearn.linear_model import RidgeCV

# visualization
import matplotlib.pyplot as plt
%matplotlib inline

# augmentation
from albumentations.pytorch import ToTensorV2
import albumentations as A

# deep learning
import timm
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR, CosineAnnealingLR, ReduceLROnPlateau, StepLR, LambdaLR
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import imageio
from PIL import Image
from tqdm.notebook import tqdm
tqdm.pandas()

# metrics
from sklearn.metrics import root_mean_squared_error

In [ ]:
class Config:
    model_name = "beit"#"swint_large224"
    base_dir = "/content/drive/MyDrive/petfinder"
    data_dir = "../input/petfinder-pawpularity-score/"
    meta_data_dir = "../input/trainmeta/"
    model_dir = "."
    output_dir = "."
    img_train_dir = os.path.join(data_dir, "train")
    img_test_dir = os.path.join(data_dir, "test")
    random_seed = 555
    n_epoch = 5
    n_fold = 5
    tta = True # calculate cv score in case TTA is executed
    tta_times = 4
    tta_beta = 1 / tta_times
    model_path = 'beit_base_patch16_224.in22k_ft_in22k_in1k'#"swin_large_patch4_window7_224"
    pretrained = True
    inp_channels = 3
    im_size =  224
    lr = 2e-5
    opt_wd_non_norm_bias = 0.01
    opt_wd_norm_bias = 0
    opt_beta1 = 0.9
    opt_beta2 = 0.99
    opt_eps = 1e-5
    batch_size = 16
    epoch_step_valid = 3
    steps_per_epoch = 62
    num_workers = 8
    out_features = 1
    dropout = 0
    aug_decay_epoch = 4
    mixup = False
    if mixup:
        mixup_epoch = n_epoch
    else:
        mixup_epoch = 0
    mixup_alpha =0.2
    scheduler_name = "OneCycleLR" #OneCycleLR
    reduce_lr_factor = 0.6
    reduce_lr_patience = 1
    T_0 = 4
    T_max =4
    T_mult =1
    min_lr = 1e-7
    max_lr =2e-5
    is_debug = False
    if is_debug:
        n_epoch = 1
        aug_decay_epoch = 1
        n_fold = 2
        n_sample_debug = 500
        tta_times = 2
        tta_beta = 1 / tta_times

In [ ]:
def seed_everything(seed=Config.random_seed):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False

seed_everything()
# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'Using device: {device}')

Using device: cpu


### Prepare data

In [ ]:
class PetDataset(Dataset):
    def __init__(self, img_dir,df, is_train,transform=None,scaler=None):
        self.img_dir = img_dir
        self.img_paths = None
        self.paths = []
        for dirname, _, filenames in os.walk(petfinder_pawpularity_score_path):
          for filename in filenames:
            folder = dirname.split('/')[-1]
            if(filename.endswith('.jpg') and folder == 'train'):
              self.paths.append(os.path.join(dirname, filename))
        if is_train:
            self.img_paths =self.paths[:8920]
        else:
            self.img_paths = self.paths[8920:]
        self.df = df.set_index(['Id'])
        self.transform = transform
        self.labels = [self.df.loc[i.split('.')[1].split('/')[-1]]['Pawpularity'] for i in self.img_paths]
        self.labels_scaled = torch.tensor(self.labels)/100

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img_id = img_path.split('.')[0].split('/')[-1]
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        if self.transform:
            image = self.transform(image = image)["image"]
        image = image / 255 # convert to 0-1
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return torch.tensor(image, dtype = torch.float), self.labels_scaled[idx].float()

    def get_unscaled_labels(self):
        return self.labels

    def get_scaled_labels(self):
        return self.labels_scaled

    def get_scaler(self):
        return self.scaler

In [ ]:
class PetDatasetTest(Dataset):
    def __init__(self, img_dir,df, is_train,transform=None,scaler=None):
        self.img_dir = img_dir
        #self.img_paths = None
        self.img_paths = []
        for dirname, _, filenames in os.walk(petfinder_pawpularity_score_path):
          for filename in filenames:
            folder = dirname.split('/')[-1]

            if(filename.endswith('.jpg') and folder == 'test'):
              self.img_paths.append(os.path.join(dirname, filename))
        self.df = df.set_index(['Id'])
        self.labels = [self.df.loc[i.split('.')[1].split('/')[-1]]['Face'] for i in self.img_paths]
        self.ids = [i.split('.')[1].split('/')[-1] for i in self.img_paths]
        self.scaler = scaler
        self.labels_scaled = torch.tensor(self.labels)/100

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img_id = img_path.split('.')[0].split('/')[-1]
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        if self.transform:
            image = self.transform(image = image.numpy())["image"]
        image = image / 255 # convert to 0-1
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return  torch.tensor(image, dtype = torch.float), self.labels[idx]

    def get_unscaled_labels(self):
        return self.labels

    def get_scaled_labels(self):
        return self.labels_scaled

    def get_scaler(self):
        return self.scaler

    def get_ids(self):
        return self.ids

In [ ]:
#IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
#IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB
def get_train_transforms(epoch=0, dim = Config.im_size):
    return A.Compose(
        [
            # unused transforms are commented out
            A.SmallestMaxSize(max_size=dim, p=1.0),
            A.RandomCrop(height=dim, width=dim, p=1.0),
            A.VerticalFlip(p = 0.5),
            A.HorizontalFlip(p = 0.5)
            #A.SafeRotate(),
            #A.CLAHE()
            #A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
        ]
  )

def get_inference_fixed_transforms(mode=0, dim = Config.im_size):
    #if mode == 0: # do not original aspects, colors and angles
    return A.Compose([
            A.SmallestMaxSize(max_size=dim, p=1.0),
            A.CenterCrop(height=dim, width=dim, p=1.0)
            #A.CLAHE()
            #A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
        ], p=1.0)

In [ ]:
traindf = None
testdf = None
for dirname, _, filenames in os.walk(petfinder_pawpularity_score_path):
    for filename in filenames:
      if(filename.endswith('train.csv')):
        traindf = pd.read_csv(os.path.join(dirname, filename))

      elif(filename.endswith('test.csv')):

        testdf = pd.read_csv(os.path.join(dirname, filename))

In [ ]:
train_transform = get_train_transforms()
val_transform = get_inference_fixed_transforms()
trainset = PetDataset(None,traindf,True,None)
valset = PetDataset(None,traindf,False,val_transform)
testset = PetDatasetTest(None,testdf,False,val_transform)

val_transform384 = get_inference_fixed_transforms(384)
valset384 = PetDataset(None,traindf,False,val_transform384)
testset384 = PetDatasetTest(None,testdf,False,val_transform384)

In [ ]:
#Config.batch_size is set to 16 for 224 image size and 8 for 384 image size
train_loader = DataLoader(
          trainset,
          batch_size = Config.batch_size,
          shuffle = True,
          num_workers = Config.num_workers,
          pin_memory = True
        )
val_loader = DataLoader(
      valset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )

val_loader384 = DataLoader(
      valset384,
      batch_size = 8,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )

#Batch size of 1 for testing due to kaggle submission portal
test_loader = DataLoader(
      testset,
      batch_size = 1,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )

test_loader384 = DataLoader(
      testset384,
      batch_size = 1,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )

### Training Utilities

In [ ]:
def get_scheduler(optimizer):
    scheduler = None
    if Config.scheduler_name == 'CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(
            optimizer,
            T_0=Config.T_0,
            eta_min=Config.min_lr,
            last_epoch=-1
        )
    elif Config.scheduler_name == 'OneCycleLR':
        scheduler = OneCycleLR(
            optimizer,
            max_lr=Config.max_lr,
            pct_start = 0.25, # same as fastai, defaut 0.3
            steps_per_epoch=int(len(train_loader)),
            epochs = Config.n_epoch
        )

    elif Config.scheduler_name == 'CosineAnnealingLR':
        scheduler = CosineAnnealingLR(
            optimizer,
            T_max=Config.T_max,
            eta_min=Config.min_lr,
            last_epoch=-1
        )
    elif Config.scheduler_name == 'ReduceOnPlateauLR':
        scheduler = ReduceLROnPlateau(
            optimizer,
            mode = 'min',
            factor=Config.reduce_lr_factor,
            patience=Config.reduce_lr_patience,
            verbose = True
        )
    return scheduler

In [ ]:
def usr_rmse_score(output, target):
    y_pred = torch.sigmoid(output).cpu()
    y_pred = y_pred.detach().numpy()*100
    target = target.cpu()*100

    return root_mean_squared_error(target, y_pred)

def rmse_oof(_oof_df, fold=None):
    oof_df = _oof_df.copy()
    if fold is not None:
        oof_df = oof_df[oof_df["fold"] == fold]
    target = oof_df['Pawpularity'].values
    y_pred = oof_df['pred'].values
    if fold is not None:
        print(f'fold {fold}: {root_mean_squared_error(target, y_pred)}')
    else:
        print(f'overall: {root_mean_squared_error(target, y_pred)}')

class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"],
                    float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )



In [ ]:
def valid_fn(val_loader, model, criterion, epoch):
    model.eval()
    #model = layer_freeze(model)
    tta_mode = 0
    test_targets = []
    test_preds = []

    metric_monitor = MetricMonitor()
    stream = tqdm(val_loader)
    for i, (images, target) in enumerate(stream, start = 1):
        images = images.to(device, non_blocking = True).float()
        target = target.to(device, non_blocking = True).float().view(-1, 1)
        with torch.no_grad():
            output = model(images)
        loss = criterion(output, target)
        rmse_score = usr_rmse_score(output, target)
        metric_monitor.update('Loss', loss.item())
        metric_monitor.update('RMSE', rmse_score)
        stream.set_description(f"Epoch: {epoch:02}. Valid. {metric_monitor}")

        targets = (target.detach().cpu().numpy() * 100).ravel().tolist()
        pred = (torch.sigmoid(output).detach().cpu().numpy() * 100).ravel().tolist()

        test_preds.extend(pred)
        test_targets.extend(targets)
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    #del val_loader, valid_dataset, target, output
    #gc.collect()
    torch.cuda.empty_cache()
    return test_targets, test_preds

### PyTorch Models

In [ ]:
# Model used for the concat method as described in the paper
class PetNet(nn.Module):
    def __init__(
        self,
        model_name1 = Config.model_path1,
        model_name2 = Confit.model_path2,
        out_features = Config.out_features,
        inp_channels=Config.inp_channels,
        pretrained=Config.pretrained
    ):
        super().__init__()
        self.swin = timm.create_model(model_name1, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)
        self.beit = timm.create_model(model_name2, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)
        self.linear1 = nn.Linear(8,8)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(8,1)

    def forward(self, image):
        swin_out = self.swin(image)
        beit_out = self.beit(image)
        out = torch.cat((swin_out, beit_out), 0)
        out = self.linear1(out)
        out = self.relu(out)
        out = self.linear2(out)
        return out


'\nclass PetNet(nn.Module):\n    def __init__(\n        self,\n        model_name1 = Config.model_path1,\n        model_name2 = Confit.model_path2,\n        out_features = Config.out_features,\n        inp_channels=Config.inp_channels,\n        pretrained=Config.pretrained\n    ):\n        super().__init__()\n        self.swin = timm.create_model(model_name1, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)\n        self.beit = timm.create_model(model_name2, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)\n        self.linear1 = nn.Linear(8,8)\n        self.relu = nn.ReLU()\n        self.linear2 = nn.Linear(8,1)\n    \n    def forward(self, image):\n        swin_out = self.swin(image)\n        beit_out = self.beit(image)\n        out = torch.cat((swin_out, beit_out), 0)\n        out = self.linear1(out)\n        out = self.relu(out)\n        out = self.linear2(out)\n        return out\n'

In [ ]:
#PyTorch wrapper used for training individual models
class PetNet(nn.Module):
    def __init__(
        self,
        model_name,
        out_features = Config.out_features,
        inp_channels=Config.inp_channels,
        pretrained=Config.pretrained
    ):
        super().__init__()
        #load model from Timm library
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)

    def forward(self, image):
        output = self.model(image)
        return output

### Training

In [ ]:
#initialize model, optimizer and loss function
model = PetNet("swin_large_patch4_window12_384")
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), Config.lr)
scheduler = get_scheduler(optimizer)

model.safetensors:   0%|          | 0.00/801M [00:00<?, ?B/s]

In [ ]:
def training_loop(train_loader,val_loader):
    #skf = StratifiedKFold(n_splits = Config.n_fold, shuffle=True, random_state=Config.random_seed)
    oof_df = pd.DataFrame()
    best_rmse = np.inf
    scaler = GradScaler()
    val_errors = []
    train_errors = []
    for epoch in range(1, Config.n_epoch + 1):
        print(f'=== epoch: {epoch}: training ===')

        metric_monitor = MetricMonitor()
        stream = tqdm(train_loader)

        for batch_idx, (images, target) in enumerate(stream, start = 1):
            model.train()
            images = images.to(device, non_blocking = True).float()
            target = target.to(device, non_blocking = True).float().view(-1, 1)
            optimizer.zero_grad()
            with autocast(): # mixed precision
                output = model(images)
                loss = criterion(output, target)
            rmse_score = usr_rmse_score(output, target)
            metric_monitor.update('Loss', loss.item())
            metric_monitor.update('RMSE', rmse_score)
            stream.set_description(f'Epoch: {epoch:02}. Train. {metric_monitor}')
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (scheduler is not None) & (Config.scheduler_name != 'ReduceOnPlateauLR') :
                scheduler.step()

            if ( ( ( batch_idx % Config.steps_per_epoch == 0) & (epoch >= Config.epoch_step_valid) ) | ( batch_idx == len(train_loader) ) ):
                valid_targets, preds = valid_fn(val_loader, model, criterion, epoch)
                valid_rmse = round(root_mean_squared_error(valid_targets, preds), 3)
                val_errors.append(valid_rmse)
                train_errors.append(rmse_score)
                print(f'epoch: {epoch}, batch: {batch_idx}/{len(train_loader)}, valid rmse: {valid_rmse}')
                if Config.scheduler_name == 'ReduceOnPlateauLR':
                    scheduler.step(valid_rmse)

                if valid_rmse < best_rmse:
                    best_rmse = valid_rmse
                    model_name = Config.model_path
                    torch.save(model.state_dict(), f'{Config.model_dir}/eva{epoch}.pth')
                    print("saved model.")
                    _oof_df = pd.DataFrame(data={ 'pred':preds,  'Pawpularity':valid_targets})

    torch.cuda.empty_cache()
    oof_df = pd.concat([oof_df, _oof_df])
    return oof_df,train_errors,val_errors#.sort_values('Id')

In [ ]:
oof_df,t_err,v_err = training_loop(train_loader, val_loader)

=== epoch: 1: training ===


  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/124 [00:00<?, ?it/s]

epoch: 1, batch: 1115/1115, valid rmse: 17.2
saved model.
=== epoch: 2: training ===


  0%|          | 0/1115 [00:00<?, ?it/s]

...

  0%|          | 0/124 [00:00<?, ?it/s]

epoch: 4, batch: 806/1115, valid rmse: 17.585


  0%|          | 0/124 [00:00<?, ?it/s]

epoch: 4, batch: 868/1115, valid rmse: 17.489


KeyboardInterrupt: 

### Load Models for Evaluation

In [ ]:
sarahpike1_timm_fine_tuned_4_path = kagglehub.dataset_download('sarahpike1/timm-fine-tuned-4')
sarahpike1_beit_fine_tuned_4_path = kagglehub.dataset_download('sarahpike1/beit-fine-tuned-4')
sarahpike1_swin_beit_ensemble_path = kagglehub.dataset_download('sarahpike1/swin-beit-ensemble')
sarahpike1_conv_fine_tuned_4_path = kagglehub.dataset_download('sarahpike1/conv-fine-tuned-4')
sarahpike1_eva02_fine_tuned_4_path = kagglehub.dataset_download('sarahpike1/eva02-fine-tuned-4')
sarahpike1_beit_fine_tuned_4_v1_path = kagglehub.dataset_download('sarahpike1/beit-fine-tuned-4-v1')
sarahpike1_swin384_fine_tuned_path = kagglehub.dataset_download('sarahpike1/swin384-fine-tuned')

In [ ]:
model1 = PetNet('convnext_large.fb_in1k')
model1.load_state_dict(torch.load(sarahpike1_conv_fine_tuned_4_path , weights_only=True))
model1 = model1.to(device)
model1.eval()

model2 = PetNet('eva02_base_patch14_224')
model2.load_state_dict(torch.load(sarahpike1_eva02_fine_tuned_4_path, weights_only=True,map_location=torch.device('cpu')))
model2 = model2.to(device)
model2.eval()

model3 = PetNet("beit_base_patch16_224")
model3.load_state_dict(torch.load(sarahpike1_beit_fine_tuned_4_v1_path, weights_only=True,map_location=torch.device('cpu')))
model3 = model3.to(device)
model3.eval()

model4 = PetNet("swin_large_patch4_window7_224")
model4.load_state_dict(torch.load(sarahpike1_timm_fine_tuned_4_path, weights_only=True,map_location=torch.device('cpu')))
model4 = model4.to(device)
model4.eval()

model5 = PetNet("swin_large_patch4_window12_384")
model5.load_state_dict(torch.load(sarahpike1_swin384_fine_tuned_path, weights_only=True,map_location=torch.device('cpu')))
model5 = model5.to(device)
model5.eval()

### Get Predictions on Validation Set

In [ ]:
def get_preds(val_loader, model):
    model.eval()
    tta_mode = 0
    ids = []
    test_preds = []

    stream = tqdm(val_loader)
    for i, (images,id_) in enumerate(stream, start = 1):
        images = images.to(device, non_blocking = True).float()
        with torch.no_grad():
            output = model(images)
        pred = (torch.sigmoid(output).detach().cpu().numpy() * 100).ravel().tolist()
        test_preds.extend(pred)
        ids.append(id_)
    test_preds = np.array(test_preds)
    torch.cuda.empty_cache()
    return test_preds,ids

In [ ]:
preds1,_ = get_preds(val_loader,model1)
preds2,_ = get_preds(val_loader,model2)
preds3,_ = get_preds(val_loader,model3)
preds4,_ = get_preds(val_loader,model4)
preds5,_ = get_preds(val_loader384,model5)

### Training SVR

In [ ]:
#Reshape and concat predictions from each model
p1 = preds1.reshape(1,-1)
p2 = preds2.reshape(1,-1)
p3 = preds3.reshape(1,-1)
p4 = preds4.reshape(1,-1)
p5 = preds5.reshape(1,-1)
p = np.concatenate((p1,p2,p3,p4,p5),axis=0)
p = p.T / 100
p.shape

In [ ]:
svr_lin = SVR(kernel = 'linear')
svr_lin.fit(p,valset.get_scaled_labels())

### Get Test Predictions

Prediction using mean ensemble method

In [ ]:
preds1,ids = get_preds(test_loader,model1)
preds2,_ = get_preds(test_loader,model2)
preds3,_ = get_preds(test_loader,model3)
preds4,_ = get_preds(test_loader,model4)
preds5,_ = get_preds(test_loader384,model5)


result = np.mean([preds1,preds2,preds3,preds4,preds5],axis=0)

ids_series = pd.Series(testset.get_ids(), name='Id')
pawpularity = pd.Series(result, name='Pawpularity')

# Concatenate the Series to form a DataFrame
df = pd.concat([ids_series,pawpularity], axis=1)
df.to_csv('submission.csv', index=False)

Prediction using SVR ensemble method

In [ ]:
preds1,ids = get_preds(test_loader,model1)
preds2,_ = get_preds(test_loader,model2)
preds3,_ = get_preds(test_loader,model3)
preds4,_ = get_preds(test_loader,model4)
preds5,_ = get_preds(test_loader384,model5)

p1 = preds1.reshape(1,-1)
p2 = preds2.reshape(1,-1)
p3 = preds3.reshape(1,-1)
p4 = preds4.reshape(1,-1)
p5 = preds5.reshape(1,-1)
p = np.concatenate((p1,p2,p3,p4,p5),axis=0)
p = p.T / 100

result = svr_lin.predict(p)*100
ids_series = pd.Series(testset.get_ids(), name='Id')
pawpularity = pd.Series(result, name='Pawpularity')

# Concatenate the Series to form a DataFrame
df = pd.concat([ids_series,pawpularity], axis=1)
df.to_csv('submission.csv', index=False)

Prediction using Linear Regression ensemble method

In [ ]:
preds1,ids = get_preds(test_loader,model1)
preds2,_ = get_preds(test_loader,model2)
preds3,_ = get_preds(test_loader,model3)
preds4,_ = get_preds(test_loader,model4)
preds5,_ = get_preds(test_loader384,model5)
#coeffs learned with statsmodels package on validation set
coefs = np.array([-4.39375493,  0.16096361,  0.1716341 ,  0.28353361,  0.01015028,0.4839837 ])
p_arr = np.vstack((np.ones(len(preds1)),preds1,preds2,preds3,preds4,preds5))
result = np.matmul(p_arr.T, coefs)
ids_series = pd.Series(testset.get_ids(), name='Id')
pawpularity = pd.Series(result, name='Pawpularity')

# Concatenate the Series to form a DataFrame
df = pd.concat([ids_series,pawpularity], axis=1)
df.to_csv('submission.csv', index=False)

### Works Cited

Wightman, R. (2019). PyTorch Image Models.

Ytakayama. (2022, January 3). [train]pytorch swin+5fold+some tips. Kaggle. https://www.kaggle.com/code/ytakayama/train-pytorch-swin-5fold-some-tips